## **Quadratic module** 
Notebook to test the algorithm for representation in the quadratic module (and archimedeanity).

In [73]:
import itertools, time, cvxpy as cp, numpy as np, sympy as sp, symengine as se
from sympy import symbols, expand
x1, x2, x3 = symbols('x1 x2 x3')

In [74]:
def sdp(matrix, threshold=1e-4): 
    if matrix.shape[0] != matrix.shape[1]: 
        return False
    
    eigenvalues, _ = np.linalg.eig(matrix)
    eigenvalues[np.abs(eigenvalues) < threshold] = 0

    print(eigenvalues)
    return np.all(eigenvalues >= 0)

In [75]:
def decomposition(matrix):
    no_rep_monom = set()
    desc_matrices = list()

    for row in matrix:
        for element in row:
            no_rep_monom.add(tuple(element))

    for mon in no_rep_monom:
        desc_matrix = []
        for row in matrix:
            new_row = []
            for element in row:
                if tuple(element) == mon:
                    new_row.append(1)
                else:
                    new_row.append(0)
            desc_matrix.append(new_row)

        desc_matrices.append(desc_matrix)

    return list(no_rep_monom), desc_matrices

In [76]:
def expand_expression(matrix, vd):
    matrix = np.array(matrix)
    
    num_vars = len(vd[0])
    variables = sp.symbols(f'x1:{num_vars+1}')
    
    variable_terms = []
    for exponents in vd:
        if all(e == 0 for e in exponents):
            term = 1  # (0,0) is 1
        else:
            term = sp.Mul(*(variables[j]**exponents[j] for j in range(num_vars) if exponents[j] != 0))
        variable_terms.append(term)
    
    expression = 0
    for col in range(matrix.shape[1]):
        term = 0
        for row in range(matrix.shape[0]):
            term += matrix[row, col] * variable_terms[row]  
        expression += term**2  

    return se.expand(expression)

In [77]:
def prod_decomposition(Q_matrices):
    desc = list()
    for matrix in Q_matrices:
        U, s, Vt = np.linalg.svd(matrix) 
        s_sqrt = np.sqrt(s)
        S_sqrt = np.diag(s_sqrt)
        temp_H = np.dot(U, S_sqrt)
        desc.append(temp_H)
        
        """
        # Check that Q = H * H^T
        reconstructed_Q = np.dot(temp_H, temp_H.T)
        
        print("\nReconstructed Q:")
        print(reconstructed_Q)
        print("\nH:")
        print(temp_H)
        """
        
    return desc

In [78]:
def check_output(polynomial, solver):
    if solver == 'SCS':
        tol = 0.00001
        
    elif solver == 'MOSEK':
        tol = 0.00000001
            
    f_monomials = [term[1] for term in f]
    out_result = 1
    for term in f:
        found = False
        for element in polynomial:
            if element[1] == term[1]:
                found = True
                if abs(element[0]-term[0]) >= tol:
                    out_result = 0
                break
        
        if out_result == 0:
            break

        if found == False:
            print(term)
            out_result = 0
            break
    
    for element in polynomial:
        if element[1] not in f_monomials and abs(element[0]) >= tol: 
            out_result = 0
            break

    return out_result

In [79]:
def generate_monomials(n_variables, deg):
    monomials = []
    for comb in itertools.product(range(deg + 1), repeat=n_variables):
        if sum(comb) <= deg and comb not in monomials:
            if n_variables == 1:
                monomials.append((comb[0], 0))
            else:
                monomials.append(comb)
    return monomials

In [90]:
"""
We can choose the solver: SCS or MOSEK.
"""
def QM(f, g, vd, solver):
    Q = []
    constraints = []
    # Create the unknown matrices Qᵢ and impose the SDP constraints
    for i in range(len(g)): 
        exec(f"Q{i} = cp.Variable((len(vd[i]), len(vd[i])), symmetric=True)")
        exec(f"Q.append(Q{i})")
        exec(f"constraints.append(Q{i} >> 0)")
    
    prods_vd = list()
    for vd_monomials in vd:
        # Matrix of monomials resulting from vd * vd'
        prod_vd = [[tuple(a[i] + c[i] for i in range(len(vd_monomials[0]))) for c in vd_monomials] for a in vd_monomials] 
        prods_vd.append(prod_vd)

    final_expression = list()
    for idx, prod in enumerate(prods_vd):
        fi = list() # We construct fi = vdᵢ' * Qi * vdᵢ * gᵢ
        l1, l2 = decomposition(prod)

        if idx == 0:
            for i in range(len(l2)):
                fi.append((cp.trace(Q[idx] @ l2[i]), l1[i]))
                
        else:
            for coef, mon in g[idx]:
                for i in range(len(l2)):
                    fi.append(( coef * cp.trace(Q[idx] @ l2[i]), tuple(sum(pair) for pair in zip(l1[i], mon)) ))

        final_expression.append(fi)

    diff_monomials = []
    for prod in final_expression:
        for coef, mon in prod: 
            if mon not in diff_monomials:
                diff_monomials.append(mon)

    # We store the coefficients of f in the same order as the different monomials that appear in the decomposition
    f_coef = []
    for monomial in diff_monomials:
        is_in_poly = False
        for coef, mon in f:
            if monomial == mon:
                f_coef.append(coef)
                is_in_poly = True
                continue
        if is_in_poly == False: f_coef.append(0)    

    
    # For each possible monomial, we go through the expanded products to take the corresponding coefficient and impose the constraints
    for idx, monomial in enumerate(diff_monomials):
        coef_sum = 0
        for prod in final_expression: # We are taking the coefficients of each monomial and accumulating them for each gᵢ
            for coef, mon in prod: 
                if mon == monomial:
                    coef_sum += coef
                    continue    
                    
        constraints += [ coef_sum == f_coef[idx]]
    
    prob = cp.Problem(cp.Minimize(0), constraints)

    try:
        if solver == 'SCS':
            prob.solve(solver='SCS', verbose=False)
            
        elif solver == 'MOSEK':
            prob.solve(solver='MOSEK', verbose=False, mosek_params = {'MSK_IPAR_NUM_THREADS':  3})
        
        Q_values = []
        for i, Qi in enumerate(Q):
            if Qi.value is not None:
                #print(sdp(Qi.value))
                #print(f"Q{i}:")
                #print(Qi.value)
                #print("..............................")
                Q_values.append(Qi.value)
            else:
                print("No solution found.")


    except cp.error.SolverError as e:
        print("Error with the solver:", e)
        Q_values = [None for i in range(len(g))]

    return Q_values

In [87]:
"""
Method responsible for the general logic
"""
def main(f, g, vd, solver):
    univariate = all(len(monomial) == 2 and monomial[1] == 0 for _, monomial in f)
    
    Qs = QM(f, g, vd, solver)

    if all(q is not None for q in Qs):
        Hs = prod_decomposition(Qs)
        final_expression = 0

        # ---------------- It has to be changed by the user according to the input (it could be read from g) ----------------
        """
        g_expression = [1, x1**3-x2**2, 1-x1] # Example 1
        g_expression = [1, x1**4, x1**2] # Example 2
        g_expression = [1, x1-1/2, x2-1/2, 1-x1*x2] # Example 3
        g_expression = [1, x1, x2, 1-x1-x2] # Example 4
        g_expression = [1, x1*x3**2+1-x1**2-x2**2, -x1*x3**2+1] # Example 5
        g_expression = [1, x1**2+x2**2, x1, x2] # Example 6 and 7
        """
        g_expression = [1, x1**2+x2**2, x1, x2] 
        
        for idx, H in enumerate(Hs):
            final_expression += expand_expression(Hs[idx], vd[idx]) * g_expression[idx]
        
        expr = sp.expand(final_expression)
        terms = expr.as_ordered_terms()
        variables = expr.free_symbols
        variables = sorted(variables, key=lambda x: str(x))  

        result = []
        for term in terms:
            coef, mon = term.as_coeff_Mul()  
            if mon == 1:  
                tuple_mon = (0,) * len(variables)
            else:
                tuple_mon = tuple(mon.as_powers_dict().get(var, 0) for var in variables)

            if univariate: # If we are in the univariate case, we need to properly format the polynomial structure
                tuple_mon = (tuple_mon[0], 0)
            result.append((coef, tuple_mon))

        if len(result[0][1]) == 0:
            out_result = -1 # No solution found
        
        else:
            out_result = check_output(result, solver) # 0: solution not accepted as valid, 1: accepted
        
        print(out_result)
        print("------------------------")
        print(expr)

#### **Example 1**
The polynomial $f(x_1, x_2) = 2-x_1^2-x_2^2$, and the archimedean quadratic module generated by $g_1(x_1, x_2) = x_1^3-x_2^2$, and $g_2(x_1, x_2) = 1-x_1$.

In [84]:
# ---------------- User input ----------------
f = [(2,(0, 0)), (-1,(2, 0)), (-1,(0, 2))] 
g0 = []
g1 = [(1,(3, 0)), (-1,(0, 2))] 
g2 = [(1,(0, 0)), (-1,(1, 0))] 
g = [g0, g1, g2] 
vd = [generate_monomials(2,1), generate_monomials(2,1), generate_monomials(2,1)]
# -------------------------------------------------
main(f, g, vd, 'SCS')

1
------------------------
5.18933191866327e-19*x1**5 - 1.8439775228394e-37*x1**4*x2 + 1.014788095028e-9*x1**4 + 1.5925029820368e-39*x1**3*x2**2 - 3.63420991266406e-28*x1**3*x2 + 4.48967529820266e-10*x1**3 - 5.18933191866327e-19*x1**2*x2**2 + 3.18556619081831e-17*x1**2*x2 - 0.999999999637339*x1**2 + 1.8439775228394e-37*x1*x2**3 - 3.27200037354217e-9*x1*x2**2 - 3.03578987616946e-17*x1*x2 - 3.37154082430402e-10*x1 - 1.5925029820368e-39*x2**4 + 3.63420991266406e-28*x2**3 - 0.999998866663319*x2**2 + 7.60719409864751e-19*x2 + 1.99999999940002


In [85]:
# ---------------- User input ----------------
f = [(2,(0, 0)), (-1,(2, 0)), (-1,(0, 2))] 
g0 = []
g1 = [(1,(3, 0)), (-1,(0, 2))] 
g2 = [(1,(0, 0)), (-1,(1, 0))] 
g = [g0, g1, g2] 
vd = [generate_monomials(2,1), generate_monomials(2,1), generate_monomials(2,1)]
# -------------------------------------------------
main(f, g, vd, 'MOSEK')

1
------------------------
2.3558704913714e-10*x1**5 + 4.33265081385978e-10*x1**4 + 3.66844980104332e-11*x1**3*x2**2 + 1.37194322569201e-9*x1**3 - 2.3558704913714e-10*x1**2*x2**2 - 0.999999997690839*x1**2 - 1.34976699858919e-9*x1*x2**2 + 1.07391395776091e-9*x1 - 3.66844980104332e-11*x2**4 - 0.999999991247321*x2**2 + 1.99999999535579


#### **Example 2**
The polynomial $f(x_1) = 2x_1^6+2x_1^5+x_1^4+x_1^2$, and the quadratic module generated by $g_1(x_1) = x_1^4$, and $g_2(x_1) = x_1^2$.

In [22]:
# ---------------- User input ----------------
f = [(2,(6,0)), (2,(5, 0)), (1,(4, 0)), (1,(2, 0))]
g0 = []
g1 = [(1,(4, 0))] 
g2 = [(1,(2, 0))]
g = [g0, g1, g2] 
vd = [generate_monomials(1, 1), generate_monomials(1, 1), generate_monomials(1, 1)]
# -------------------------------------------------
main(f, g, vd, 'SCS')

1
------------------------
2.00000004565287*x1**6 + 2.00000004601902*x1**5 + 1.0000000132839*x1**4 + 1.0000000135163*x1**2


In [23]:
# ---------------- User input ----------------
f = [(2,(6,0)), (2,(5, 0)), (1,(4, 0)), (1,(2, 0))]
g0 = []
g1 = [(1,(4, 0))] 
g2 = [(1,(2, 0))] 
g = [g0, g1, g2] 
vd = [generate_monomials(1, 1), generate_monomials(1, 1), generate_monomials(1, 1)]
# -------------------------------------------------
main(f, g, vd, 'MOSEK')

1
------------------------
1.99999999999921*x1**6 + 1.99999999999939*x1**5 + 0.999999999999342*x1**4 + 0.99999999999961*x1**2 + 1.53199047334635e-13


#### **Example 3**
The polynomial $f(x_1, x_2) = 4.5-x_1^2-x_2^2$, and the non-archimedean quadratic module generated by $g_1(x_1, x_2) = x_1-1/2$, $g_2(x_1, x_2) = x_2-1/2$, and $g_3(x_1, x_2) = 1-x_1x_2$.

In [30]:
# ---------------- User input ----------------
f = [(4.5,(0, 0)), (-1,(2, 0)), (-1,(0, 2))] 
g0 = []
g1 = [(1,(1, 0)), (-1/2,(0, 0))] 
g2 = [(1,(0, 1)), (-1/2,(0, 0))] 
g3 = [(1,(0, 0)), (-1,(1, 1))]
g = [g0, g1, g2, g3] 
vd = [generate_monomials(2, 4), generate_monomials(2, 4), generate_monomials(2, 4), generate_monomials(2, 3)]
# -------------------------------------------------
main(f, g, vd, 'SCS')

0
------------------------
0.000136707943356487*x1**9 + 0.000111923893855645*x1**8*x2 - 0.000239404032126257*x1**8 + 0.000184700477276357*x1**7*x2**2 - 0.000172959959280128*x1**7*x2 + 0.000217341458680702*x1**7 + 4.77726416640996e-6*x1**6*x2**3 - 0.000242377076887887*x1**6*x2**2 - 2.04122283467001e-5*x1**6*x2 - 7.57662519506042e-5*x1**6 + 0.000128703465278735*x1**5*x2**4 - 8.92117191207131e-5*x1**5*x2**3 - 2.9767008687287e-5*x1**5*x2**2 - 6.9651464809084e-5*x1**5*x2 - 0.000100035923180641*x1**5 + 0.000128703454674966*x1**4*x2**5 - 7.64042715810653e-5*x1**4*x2**4 + 4.32441920423798e-5*x1**4*x2**3 + 1.52212199555635e-5*x1**4*x2**2 - 1.42174934367745e-5*x1**4*x2 - 2.05228564400528e-5*x1**4 + 4.77728226211838e-6*x1**3*x2**6 - 8.9211757832941e-5*x1**3*x2**5 + 4.32442206346195e-5*x1**3*x2**4 + 6.79728560790682e-5*x1**3*x2**3 + 6.02077834539472e-5*x1**3*x2**2 + 5.40216246616154e-5*x1**3*x2 - 1.41934829684942e-5*x1**3 + 0.000184700497310083*x1**2*x2**7 - 0.000242377138451974*x1**2*x2**6 - 2.97

In [29]:
# ---------------- User input ----------------
f = [(4.5,(0, 0)), (-1,(2, 0)), (-1,(0, 2))] 
g0 = []
g1 = [(1,(1, 0)), (-1/2,(0, 0))] 
g2 = [(1,(0, 1)), (-1/2,(0, 0))] 
g3 = [(1,(0, 0)), (-1,(1, 1))]
g = [g0, g1, g2, g3] 
vd = [generate_monomials(2, 4), generate_monomials(2, 4), generate_monomials(2, 4), generate_monomials(2, 3)]
# -------------------------------------------------
main(f, g, vd, 'MOSEK')

0
------------------------
1.73326624449894e-8*x1**9 + 9.71934024779213e-9*x1**8*x2 - 9.65730203036147e-9*x1**8 + 7.72789214892354e-9*x1**7*x2**2 - 4.08636284348046e-9*x1**7*x2 - 1.27297403846281e-11*x1**7 + 7.42599047275002e-9*x1**6*x2**3 - 3.61616453144545e-9*x1**6*x2**2 - 4.84529083522034e-13*x1**6*x2 - 3.02061153867328e-12*x1**6 + 7.84221952981667e-9*x1**5*x2**4 - 4.05420230026321e-9*x1**5*x2**3 + 5.4634075041804e-13*x1**5*x2**2 - 9.8987484875579e-13*x1**5*x2 - 1.59409638267327e-12*x1**5 + 7.79423795507571e-9*x1**4*x2**5 - 1.51832326746038e-9*x1**4*x2**4 + 5.95812288395337e-12*x1**4*x2**3 - 4.86499729390744e-13*x1**4*x2**2 - 5.29909449653587e-13*x1**4*x2 - 1.07938658011619e-12*x1**4 + 7.42908112837045e-9*x1**3*x2**6 - 4.05099456757263e-9*x1**3*x2**5 + 5.80158143748122e-12*x1**3*x2**4 - 2.77722289609983e-13*x1**3*x2**3 - 2.38808972596871e-13*x1**3*x2**2 - 4.26381152607291e-13*x1**3*x2 - 4.78485306931731e-13*x1**3 + 7.708234930951e-9*x1**2*x2**7 - 3.60514817781751e-9*x1**2*x2**6 + 5.

#### **Example 4**
The polynomial $f(x_1, x_2) = 1-x_1^2-x_2^2$, and the archimedean quadratic module generated by $g_1(x_1, x_2) = x_1$, $g_2(x_1, x_2) = x_2$, and $g_3(x_1, x_2) = 1-x_1-x_2$.

In [33]:
# ---------------- User input ----------------
f = [(1,(0, 0)), (-1,(2, 0)), (-1,(0, 2))] 
g0 = []
g1 = [(1,(1, 0))] 
g2 = [(1,(0, 1))] 
g3 = [(1,(0, 0)), (-1,(1, 0)), (-1,(0, 1))] 
g = [g0, g1, g2, g3] 
vd = [generate_monomials(2, 0), generate_monomials(2, 1), generate_monomials(2, 1), generate_monomials(2, 1)]
# -------------------------------------------------
main(f, g, vd, 'SCS')

1
------------------------
-5.94494453665106e-10*x1**3 + 7.37240823944774e-10*x1**2*x2 - 1.0000000004043*x1**2 + 7.37239047587934e-10*x1*x2**2 + 1.62370961120928e-9*x1*x2 + 7.1630279485646e-10*x1 - 5.94494509176258e-10*x2**3 - 1.0000000004043*x2**2 + 7.1630412712409e-10*x2 + 1.00000096348683


In [34]:
# ---------------- User input ----------------
f = [(1,(0, 0)), (-1,(2, 0)), (-1,(0, 2))] 
g0 = []
g1 = [(1,(1, 0))] 
g2 = [(1,(0, 1))] 
g3 = [(1,(0, 0)), (-1,(1, 0)), (-1,(0, 1))] 
g = [g0, g1, g2, g3] 
vd = [generate_monomials(2, 0), generate_monomials(2, 1), generate_monomials(2, 1), generate_monomials(2, 1)]
# -------------------------------------------------
main(f, g, vd, 'MOSEK')

1
------------------------
2.01430205848396e-9*x1**3 - 6.27471408165547e-11*x1**2*x2 - 0.999999991432775*x1**2 - 6.27471408165547e-11*x1*x2**2 - 1.41269912434794e-10*x1*x2 + 1.00187635965199e-9*x1 + 2.01433147939412e-9*x2**3 - 0.999999991432693*x2**2 + 1.00190622465135e-9*x2 + 0.999999994627641


#### **Example 5**
The polynomial $f(x_1, x_2, x_3) = 2-x_1^2-x_2^2$, and the quadratic module generated by $g_1(x_1, x_2, x_3) = x_1x_3^2+1-x_1^2-x_2^2$, and $g_2(x_1, x_2, x_3) = -x_1x_3^2+1$.

In [38]:
# ---------------- User input ----------------
f = [(2,(0, 0, 0)), (-1,(2, 0, 0)), (-1,(0, 2, 0))] 
g0 = []
g1 = [(1,(1, 0, 2)), (1,(0, 0, 0)), (-1,(2, 0, 0)), (-1,(0, 2, 0))] 
g2 = [(-1,(1, 0, 2)), (1,(0, 0, 0))] 
g = [g0, g1, g2] 
vd = [generate_monomials(3, 0), generate_monomials(3, 0), generate_monomials(3, 0)]
# -------------------------------------------------
main(f, g, vd, 'SCS')

1
------------------------
-1.00000000862279*x1**2 - 2.83273715595556e-8*x1*x3**2 - 1.00000000862279*x2**2 + 2.00000015023603


In [39]:
# ---------------- User input ----------------
f = [(2,(0, 0, 0)), (-1,(2, 0, 0)), (-1,(0, 2, 0))] 
g0 = []
g1 = [(1,(1, 0, 2)), (1,(0, 0, 0)), (-1,(2, 0, 0)), (-1,(0, 2, 0))] 
g2 = [(-1,(1, 0, 2)), (1,(0, 0, 0))] 
g = [g0, g1, g2] 
vd = [generate_monomials(3, 0), generate_monomials(3, 0), generate_monomials(3, 0)]
# -------------------------------------------------
main(f, g, vd, 'MOSEK')

1
------------------------
-0.999999999993442*x1**2 + 1.74793512996985e-12*x1*x3**2 - 0.999999999993442*x2**2 + 1.99999999999394


#### **Example 6**
The polynomial $f(x_1, x_2) = x_1^2+x_1^2x_2^2+x_2^4+x_1^3+x_1x_2^2+3x_1^2x_2$, and the quadratic module generated by $g_1(x_1, x_2) = x_1^2+x_2^2$, $g_2(x_1, x_2) = x_1$, and $g_3(x_1, x_2) = x_2$.

In [88]:
# ---------------- User input ----------------
# This should work (finding a decomposition), but it only works if we lower the threshold.
f = [(1,(2, 0)), (1,(2, 2)), (1,(0, 4)), (1,(3,0)), (1,(1,2)), (3,(2,1))] 
g0 = []
g1 = [(1,(2, 0)), (1,(0, 2))] 
g2 = [(1,(1, 0))] 
g3 = [(1,(0, 1))] 
g = [g0, g1, g2, g3]
vd = [generate_monomials(2, 1), generate_monomials(2, 1), generate_monomials(2, 1), generate_monomials(2, 1)]
# -------------------------------------------------
main(f, g, vd, 'SCS')

0
------------------------
7.22482830260539e-10*x1**4 - 1.15661388180919e-12*x1**3*x2 + 1.0*x1**3 + 0.999999998977398*x1**2*x2**2 + 3.0*x1**2*x2 + 1.00000000040938*x1**2 - 1.15661388180919e-12*x1*x2**3 + 1.0*x1*x2**2 + 2.15768745027756e-8*x1*x2 + 4.56013810128629e-7*x1 + 0.999999998254915*x2**4 - 5.17528520316279e-8*x2**3 - 1.62506117319661e-7*x2**2 + 9.9231163362655e-6*x2 + 0.000127162829860992


In [89]:
# ---------------- User input ----------------
# This should work (finding a decomposition), but it only works if we lower the threshold.
f = [(1,(2, 0)), (1,(2, 2)), (1,(0, 4)), (1,(3,0)), (1,(1,2)), (3,(2,1))] 
g0 = []
g1 = [(1,(2, 0)), (1,(0, 2))] 
g2 = [(1,(1, 0))] 
g3 = [(1,(0, 1))] 
g = [g0, g1, g2, g3]
vd = [generate_monomials(2, 1), generate_monomials(2, 1), generate_monomials(2, 1), generate_monomials(2, 1)]
# -------------------------------------------------
main(f, g, vd, 'MOSEK')

0
------------------------
9.39610040226742e-10*x1**4 - 9.15873741150398e-13*x1**3*x2 + 0.999999994909206*x1**3 + 0.999999995366946*x1**2*x2**2 + 2.99999998475022*x1**2*x2 + 0.999999994890813*x1**2 - 9.15873741150398e-13*x1*x2**3 + 0.999999994905829*x1*x2**2 - 1.04985031085247e-13*x1*x2 - 1.34685878986143e-11*x1 + 0.999999994427336*x2**4 - 1.58557646179336e-11*x2**3 - 2.49101209494687e-11*x2**2 + 5.92007464717382e-11*x2 + 1.03267130883149e-8


#### **Example 7**
The polynomial $f(x_1, x_2) = x_1^5+6x_1^3x_2^2+4x_1^2x_2^3+4x_1^4x_2+x_1x_2^4+x_2^4+x_1^2x_2^2+x_1^2x_2+x_1^2$, and the quadratic module generated by $g_1(x_1, x_2) = x_1^2+x_2^2$, $g_2(x_1, x_2) = x_1$, and $g_3(x_1, x_2) = x_2$.

In [62]:
# ---------------- User input ----------------
# This should work (finding a decomposition), but it only works if we lower the threshold.
f = [(1,(5, 0)), (6,(3, 2)), (4,(2, 3)), (4,(4,1)), (1,(1,4)), (1,(0,4)), (1,(2,2)), (1,(2,1)), (1,(2,0))]
g0 = []
g1 = [(1,(2, 0)), (1,(0, 2))] 
g2 = [(1,(1, 0))] 
g3 = [(1,(0, 1))] 
g = [g0, g1, g2, g3]
vd = [generate_monomials(2, 2), generate_monomials(2, 2), generate_monomials(2, 2), generate_monomials(2, 2)]
# -------------------------------------------------
main(f, g, vd, 'SCS')

0
------------------------
1.06230324223248e-9*x1**6 - 1.57556862844242e-10*x1**5*x2 + 0.999999999999999*x1**5 + 1.00286433370625e-9*x1**4*x2**2 + 4.00000000000024*x1**4*x2 + 1.86797022294627e-10*x1**4 + 3.49318670450768e-9*x1**3*x2**3 + 5.9999999999982*x1**3*x2**2 + 2.872450055591e-10*x1**3*x2 - 5.59449916948118e-11*x1**3 + 2.77317927659438e-7*x1**2*x2**4 + 3.99999999833312*x1**2*x2**3 + 1.00000023012235*x1**2*x2**2 + 0.999999987377062*x1**2*x2 + 0.999999697258163*x1**2 + 3.65074356735192e-9*x1*x2**5 + 0.999999999992532*x1*x2**4 + 2.94858865101943e-7*x1*x2**3 - 5.41505798742903e-8*x1*x2**2 - 3.66534814781128e-7*x1*x2 - 6.04955321130874e-8*x1 + 2.77377366567964e-7*x2**6 - 7.54055395535801e-8*x2**5 + 1.00008615272619*x2**4 - 8.67753142669292e-6*x2**3 - 0.000207662766099839*x2**2 + 9.11621493845396e-6*x2 + 0.00012511253177289


In [63]:
# ---------------- User input ----------------
# This should work (finding a decomposition), but it only works if we lower the threshold.
f = [(1,(5, 0)), (6,(3, 2)), (4,(2, 3)), (4,(4,1)), (1,(1,4)), (1,(0,4)), (1,(2,2)), (1,(2,1)), (1,(2,0))]
g0 = []
g1 = [(1,(2, 0)), (1,(0, 2))] 
g2 = [(1,(1, 0))] 
g3 = [(1,(0, 1))] 
g = [g0, g1, g2, g3]
vd = [generate_monomials(2, 2), generate_monomials(2, 2), generate_monomials(2, 2), generate_monomials(2, 2)]
# -------------------------------------------------
main(f, g, vd, 'MOSEK')

0
------------------------
3.77693864686243e-9*x1**6 + 1.0926442108591e-11*x1**5*x2 + 0.999999987085843*x1**5 + 6.05144155302786e-9*x1**4*x2**2 + 3.99999994848676*x1**4*x2 - 3.53528317731389e-12*x1**4 + 6.19166859297102e-11*x1**3*x2**3 + 5.99999992272586*x1**3*x2**2 + 1.06739617145024e-12*x1**3*x2 - 1.69352084439245e-11*x1**3 + 2.73381644706959e-8*x1**2*x2**4 + 3.99999994848813*x1**2*x2**3 + 0.999999987293752*x1**2*x2**2 + 0.999999987115915*x1**2*x2 + 0.999999987098254*x1**2 + 5.09902438211192e-11*x1*x2**5 + 0.999999987106982*x1*x2**4 - 3.79252185211953e-13*x1*x2**3 - 7.22780689466074e-12*x1*x2**2 - 5.36940283901721e-14*x1*x2 - 3.22456214832921e-11*x1 + 2.50636615645305e-8*x2**6 - 2.86577757238702e-11*x2**5 + 0.999999987259403*x2**4 - 1.56559765152053e-11*x2**3 - 2.33114583409258e-11*x2**2 + 3.30097806010332e-10*x2 + 3.07424545746194e-8
